In [ ]:
# run the single model file fitted on 9:4:6 split data

# %run -i "C:/Users/LL/Desktop/RPI/24_Spring/AMLF/HW_5_SHAP_LIME/Enet_NeuralNet_v2.ipynb"

In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv('C:/users/LL/Documents/GitHub/AMLF_projects/data.csv')

df = df.drop(columns = ['Unnamed: 0'])

# According to note 30: "Therefore, to predict returns at month t+1, we use most recent monthly characteristics at the end of month t." <br>
# Hence, **shift return t+1 to serve as response: r(t+1)**.

df['r(t+1)'] = df.groupby('permno')['return'].shift(-1)

### handle missing data

# According to note 30 (bottom of p 2248): "Another issue is missing characteristics, which we replace with the cross-sectional median at each month for each stock, respectively." <br>
# Hence, calculate monthly cross-sectional median for features: **'mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr'**.

df_filled = df.copy()
for feature in ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']:
    df_filled[feature] = df_filled.groupby('Date')[feature].transform(lambda x: x.fillna(x.median()))

df_filled.isna().sum()

df.loc[:, ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']] = df_filled.loc[:,['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']]

df['Date'] = pd.to_datetime(df['Date'])

# Set the datetime column as index
df.set_index('Date', inplace=True, drop = True)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_scaled = scaler.fit_transform(df)

df_scaled = pd.DataFrame(df_scaled, columns=df.columns)


permno = df['permno'].reset_index(drop = True)

df_scaled['permno'] = permno

df_scaled.index = df.index

df_scaled_2 = df_scaled.drop(columns = [ 'permno', 'return'])


### split data

##split training, validation, and testing datasets

#training : validation : testing = 6 yr : 4yr : 9 yr <br>
#Also drop the first and last month due to the absence of r(t+1) and return(t-1)

training = df_scaled_2[:'2007-01-01'].dropna()
validation = df_scaled_2['2007-01-01':'2011-01-01']
testing = df_scaled_2['2011-01-01':].dropna()

training_combined = df_scaled_2[:'2011-01-01'].dropna()


##separate X and y

X_train = training.drop(columns = ['r(t+1)'])
y_train = training['r(t+1)']

X_val = validation.drop(columns = ['r(t+1)'])
y_val = validation['r(t+1)']

X_test = testing.drop(columns = ['r(t+1)'])
y_test = testing['r(t+1)']

X_train_combined = training_combined.drop(columns = ['r(t+1)'])
y_train_combined = training_combined['r(t+1)']

In [4]:
import keras
from keras import layers
from keras import Sequential

## performance metric

# Out-of-sample R^2. (r2_score_wo_demeaning)

import tensorflow as tf

def r2_score_wo_demeaning_nn(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - 0))
    r2 = 1 - (ss_res / ss_tot)
    return r2


## tuning custom function for neural net

def compile_and_tune_model(model, parameter_dicts, x_train, y_train, x_val, y_val):
    results = []
    
    for params in parameter_dicts:

        model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']),
                      loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)  # Using mean absolute error (mae) as metric
        
        history = model.fit(x_train, y_train, epochs=params['epoch'], batch_size=params['batch_size'],
                            validation_data=(x_val, y_val), verbose=0)
        
        # Get the metric value for the last epoch
        last_epoch_metric = history.history['r2_score_wo_demeaning_nn'][-1]  # Validation MAE for last epoch
        
        # Store results for current parameter set
        results.append({'params': params, 'val_r2_score_wo_demeaning_nn': last_epoch_metric})
    
    return results

In [5]:
## NN

# All activation functions are ReLU function <br>
# optimizer: SGD w/ learning rate shrinkage: adam <br>

# convert dataframes to numpy array

X_train_nn = np.asarray(X_train.values)
y_train_nn = np.asarray(y_train.values)

X_val_nn = np.asarray(X_val.values)
y_val_nn = np.asarray(y_val.values)

X_test_nn = np.asarray(X_test.values)
y_test_nn = np.asarray(y_test.values)

X_train_combined_nn = np.asarray(X_train_combined.values)
y_train_combined_nn = np.asarray(y_train_combined.values)

# set-up grid

nepoch_val = [25, 50, 75, 100]
lr_val = [0.05, 0.01, 0.001]
nbatch_val = [1500, 2500, 3500]

param_nn = [{'epoch': epoch, 'learning_rate': learning_rate, 'batch_size': batch_size} for epoch in nepoch_val for learning_rate in lr_val for batch_size in nbatch_val]

In [7]:
### 3-layer

input_dim = 20
layer1_n = 32
layer2_n = 16
layer3_n = 8


model_3 = Sequential([
            layers.Dense(layer1_n, input_dim = input_dim, activation='relu'),
            layers.Dense(layer2_n, activation='relu'),
            layers.Dense(layer3_n, activation='relu'),
            layers.Dense(1, activation='linear')
        ])


## tuning

result_nn3 = compile_and_tune_model(model_3, param_nn, X_train_nn, y_train_nn, X_val_nn, y_val_nn)

result_nn3


## best model**<br>

# best parameters

result_nn3 = pd.DataFrame(result_nn3).sort_values('val_r2_score_wo_demeaning_nn',ascending=False)
opt_score = result_nn3.iloc[0,1]
opt_para = result_nn3.iloc[0,0]

result_nn3.iloc[0,1]

result_nn3.iloc[0,0]

# best model

model_nn3 = Sequential([
            layers.Dense(layer1_n, activation='relu', input_dim=input_dim),
            layers.Dense(1, activation='linear')
        ])

model_nn3.compile(optimizer=keras.optimizers.Adam(learning_rate=opt_para['learning_rate']),
                      loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)

history = model_nn3.fit(X_train_combined_nn, y_train_combined_nn, epochs=opt_para['epoch'], batch_size=opt_para['batch_size'],
                            validation_data=(X_test_nn, y_test_nn), verbose=0)
        
last_epoch_metric = history.history['val_r2_score_wo_demeaning_nn'][-1]


last_epoch_metric

-0.04292283207178116

## HW 5

sort data